In [1]:
import torch

import sys
sys.path.append('/home/azhir/Dora/pytorch_lightning')
from craftsman.models.autoencoders.michelangelo_autoencoder import MichelangeloAutoencoder

/home/azhir/Dora/pytorch_lightning/craftsman/utils/checkpoint.py:39: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.


In [2]:
import yaml

# Load the YAML configuration
yaml_path = '/home/azhir/Dora/pytorch_lightning/configs/shape-autoencoder/Dora-VAE-train.yaml'
with open(yaml_path, 'r') as file:
    yaml_config = yaml.safe_load(file)

# Extract the shape model configuration
shape_model_config = yaml_config['system']['shape_model']
shape_model_config['pretrained_model_name_or_path'] = '/home/azhir/Dora/pytorch_lightning/ckpts/Dora-VAE-1.1/dora_vae_1_1.ckpt'
shape_model_config['num_latents'] = 2048

# Instantiate the model using the YAML configuration
model = MichelangeloAutoencoder(cfg=shape_model_config)
model.eval()  # Set the model to evaluation mode

Loading pretrained shape model from /home/azhir/Dora/pytorch_lightning/ckpts/Dora-VAE-1.1/dora_vae_1_1.ckpt


MichelangeloAutoencoder(
  (embedder): FourierEmbedder()
  (encoder): PerceiverCrossAttentionEncoder(
    (embedder): FourierEmbedder()
    (input_proj): Linear(in_features=54, out_features=768, bias=True)
    (input_proj1): Linear(in_features=54, out_features=768, bias=True)
    (cross_attn): ResidualCrossAttentionBlock(
      (attn): MultiheadCrossAttention(
        (c_q): Linear(in_features=768, out_features=768, bias=False)
        (c_kv): Linear(in_features=768, out_features=1536, bias=False)
        (c_proj): Linear(in_features=768, out_features=768, bias=True)
        (attention): QKVMultiheadCrossAttention()
      )
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (c_fc): Linear(in_features=768, out_features=3072, bias=True)
        (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        (gelu): GELU(approximate='none')
      )
      (ln_3): LayerNorm

In [3]:
shape_model_config['num_latents']


2048

In [4]:
import trimesh
# Generate a closed sphere using trimesh
mesh = trimesh.creation.icosphere(subdivisions=3, radius=1.0)

# Prepare the input for the encoder
vertices = torch.tensor(mesh.vertices, dtype=torch.float32).unsqueeze(0)  # [B, N, 3]
normals = torch.tensor(mesh.vertex_normals, dtype=torch.float32).unsqueeze(0)  # [B, N, 3]
input_data = torch.cat([vertices, normals], dim=-1)  # [B, N, 6]"

model.split = 'val'  # Specify the split explicitly
latents, kl_embed, posterior = model.encode(input_data, input_data, sample_posterior=True)


query shape torch.Size([1, 2048, 768])


In [12]:
decoded_latents = model.decode(kl_embed)

In [ ]:
# Example: create a dense grid over the specified bounds
def create_dense_query_grid(bounds=(-1.05, 1.05), resolution=128):
    # ... code to generate a [1, N, 3] tensor of query points ...
    return queries

queries = create_dense_query_grid()
sdf_logits = model.query(queries, decoded_latents)

In [ ]:
queries = create_dense_query_grid(bounds=(-1.05, 1.05))  # [B, N, 3]
sdf_logits = model.query(queries, decoded_latents)

In [ ]:
mesh_v_f, has_surface = model.extract_geometry(decoded_latents, octree_depth=9, save_slice_dir='/output')

KeyboardInterrupt: 